In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras

In [2]:
from mlxtend.preprocessing import minmax_scaling 

In [3]:
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

2.11.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
df = pd.read_csv('Data/cleaned.csv', index_col=0)
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72.0,35.0,169.5,33.6,0.627,50,1
1,1,85.0,66.0,29.0,102.5,26.6,0.351,31,0
2,8,183.0,64.0,32.0,169.5,23.3,0.672,32,1
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101.0,76.0,48.0,180.0,32.9,0.171,63,0
764,2,122.0,70.0,27.0,102.5,36.8,0.340,27,0
765,5,121.0,72.0,23.0,112.0,26.2,0.245,30,0
766,1,126.0,60.0,32.0,169.5,30.1,0.349,47,1


In [5]:
def to_tensor(X_train, X_test, y_train, y_test):
    X_train = tf.convert_to_tensor(X_train.values, dtype=tf.float32)
    X_test = tf.convert_to_tensor(X_test.values, dtype=tf.float32)
    y_train = tf.convert_to_tensor(y_train.values, dtype=tf.float32)
    y_test = tf.convert_to_tensor(y_test.values, dtype=tf.float32)

    return X_train, X_test, y_train, y_test


In [6]:
X, y = df.copy().drop('Outcome', axis=1), df.Outcome
X_train, X_test, y_train, y_test = to_tensor(*train_test_split(X, y, test_size=0.3))
     

In [7]:
scaled_data = minmax_scaling(df,columns=['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age'])

In [8]:
def build_model():
    model = tf.keras.Sequential([
    tf.keras.layers.Dense(8, activation='relu', input_shape=[len(scaled_data.keys())]),
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
  ])

    optimizer = tf.keras.optimizers.RMSprop(0.01)

    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

model = build_model()

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 72        
                                                                 
 dense_1 (Dense)             (None, 4)                 36        
                                                                 
 dense_2 (Dense)             (None, 1)                 5         
                                                                 
Total params: 113
Trainable params: 113
Non-trainable params: 0
_________________________________________________________________


In [10]:
EPOCHS = 1000

history = model.fit(scaled_data, df['Outcome'],epochs=EPOCHS, validation_split=0.2, verbose=2)

Epoch 1/1000
20/20 - 2s - loss: 0.6659 - accuracy: 0.6319 - val_loss: 0.6364 - val_accuracy: 0.7013 - 2s/epoch - 121ms/step
Epoch 2/1000
20/20 - 0s - loss: 0.6063 - accuracy: 0.6873 - val_loss: 0.6000 - val_accuracy: 0.6688 - 119ms/epoch - 6ms/step
Epoch 3/1000
20/20 - 0s - loss: 0.5543 - accuracy: 0.7231 - val_loss: 0.5216 - val_accuracy: 0.7532 - 112ms/epoch - 6ms/step
Epoch 4/1000
20/20 - 0s - loss: 0.5105 - accuracy: 0.7492 - val_loss: 0.5342 - val_accuracy: 0.7792 - 136ms/epoch - 7ms/step
Epoch 5/1000
20/20 - 0s - loss: 0.4794 - accuracy: 0.7785 - val_loss: 0.5248 - val_accuracy: 0.7857 - 122ms/epoch - 6ms/step
Epoch 6/1000
20/20 - 0s - loss: 0.4713 - accuracy: 0.7801 - val_loss: 0.4880 - val_accuracy: 0.8182 - 118ms/epoch - 6ms/step
Epoch 7/1000
20/20 - 0s - loss: 0.4571 - accuracy: 0.7997 - val_loss: 0.4600 - val_accuracy: 0.7662 - 86ms/epoch - 4ms/step
Epoch 8/1000
20/20 - 0s - loss: 0.4485 - accuracy: 0.7899 - val_loss: 0.4886 - val_accuracy: 0.7403 - 79ms/epoch - 4ms/step
Epo

In [11]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

,loss,accuracy,val_loss,val_accuracy,epoch
995,0.221791,0.900651,0.762688,0.876623,995
996,0.207686,0.918567,0.899254,0.837662,996
997,0.210576,0.910423,0.823072,0.863636,997
998,0.234100,0.903909,0.749414,0.857143,998
999,0.203642,0.910423,0.583501,0.863636,999


In [12]:
acc = (hist['accuracy'].tail().sum())*100/5 
val_acc = (hist['val_accuracy'].tail().sum())*100/5 

print("Training Accuracy = {}% and Validation Accuracy= {}%".format(round(acc, 2), round(val_acc, 2)))

Training Accuracy = 90.88% and Validation Accuracy= 85.97%
